## Segmenting and Clustering in Toronto part 2

### Utilize Foursquare location data

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
# download url data from Wikipedia

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
toronto_data = BeautifulSoup(source, 'lxml')
table = toronto_data.find('table')

In [ ]:
# creat a new Dataframe
column_names = ['Postalcode','Borough','Neighborhood']
toronto_df = pd.DataFrame(columns = column_names)

In [ ]:
# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        toronto_df.loc[len(toronto_df)] = row_data

In [ ]:
toronto_df = toronto_df[toronto_df['Borough'] != 'Not assigned']

print(toronto_df['Borough'].unique())

toronto_df = toronto_df.reset_index(drop=True)

toronto_df.head()

#### Drop duplicate rows

In [ ]:
toronto_df.drop_duplicates(inplace=True)

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [ ]:
for row in range(len(toronto_df.index)):
    if(toronto_df.loc[row,'Neighborhood']=='Not assigned'):
        print(toronto_df.loc[row,:])
        toronto_df[toronto_df.loc[row,'Neighborhood']=='Not assigned']=toronto_df.loc[row,'Borough']     

#### Print dataframe's dimensions using .shape

In [ ]:
toronto_df.shape

#### Retrieve longitude and latitude coordinates for each postal code

Now a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name is build, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code.

To that, I will use the geospatial data from the link below which points to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Firstly, let's define the function **get_ccords(postal_code)** that takes as input the postal code returns the geographical coordinates for each postal code (for the sake of the assignemnt).

In [ ]:
import geocoder

def get_coords(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

#### Read geospatial data

In [ ]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [ ]:
geo_df.shape

In [ ]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, toronto_df, on='Postalcode')


In [ ]:
geo_merged.head()

Re-order the columns so as they displayed in the following order **Postalcode, Borough, Neighborhood, Latitude, Longitude**

In [ ]:
geo_data=geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]

geo_merged.head()